<a href="https://colab.research.google.com/github/SiweiLuo/DL_with_Python/blob/master/CV_preprocess2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
import string 
import keras
from keras.preprocessing.text import Tokenizer
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
from keras.datasets import imdb

from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import text, sequence

from google.colab import drive

#import StringIO
import time
import sys
import csv

In [0]:
drive.mount('/content/gdrive/')

In [0]:
questions = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/CareerVillage/questions.csv')
#print('questions = ',questions)
answers = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/CareerVillage/answers.csv')
#print('answers = ',answers)

In [0]:
questions['questions_body'] = questions['questions_body'].str.replace("<p>", " ").str.replace("</p>", " ").str.replace("\n"," ")
answers['answers_body'] = answers['answers_body'].str.replace("<p>", " ").str.replace("</p>", " ").str.replace("\n"," ")
plt_data_questions = questions['questions_body']
plt_data_answers = answers['answers_body']
plt_data_questions_id = questions['questions_id']
plt_data_answers_questions_id = answers['answers_question_id']
plt_data_answers_author_id = answers['answers_author_id']
#plt_data = pd.DataFrame([plt_data_questions, plt_data_answers,plt_data_questions_id,plt_data_answers_questions_id]).T

In [143]:
plt_data_questions.head(5)

0    What  is  a  maths  teacher?   what  is  a  ma...
1    I am Priyanka from Bangalore . Now am in 10th ...
2    I'm planning on going abroad for my first job....
3    i hear business management is a hard way to ge...
4    I'm trying to find scholarships for first year...
Name: questions_body, dtype: object

In [144]:
plt_data_answers.head(5)

0     Hi!   You are asking a very interesting quest...
1     Hi. I joined the Army after I attended colleg...
2     Dear Priyanka,  Greetings! I have answered th...
3     I work for a global company who values highly...
4    I agree with Denise. Every single job I've had...
Name: answers_body, dtype: object

In [145]:
plt_data_questions_id.head(5)

0    332a511f1569444485cf7a7a556a5e54
1    eb80205482e4424cad8f16bc25aa2d9c
2    4ec31632938a40b98909416bdd0decff
3    2f6a9a99d9b24e5baa50d40d0ba50a75
4    5af8880460c141dbb02971a1a8369529
Name: questions_id, dtype: object

In [0]:
df1 = pd.DataFrame([plt_data_questions_id,plt_data_questions]).T
df2 = pd.DataFrame([plt_data_answers_questions_id,plt_data_answers,plt_data_answers_author_id]).T

In [147]:
df1.head(5)

,questions_id,questions_body
0,332a511f1569444485cf7a7a556a5e54,What is a maths teacher? what is a ma...
1,eb80205482e4424cad8f16bc25aa2d9c,I am Priyanka from Bangalore . Now am in 10th ...
2,4ec31632938a40b98909416bdd0decff,I'm planning on going abroad for my first job....
3,2f6a9a99d9b24e5baa50d40d0ba50a75,i hear business management is a hard way to ge...
4,5af8880460c141dbb02971a1a8369529,I'm trying to find scholarships for first year...


In [148]:
df2.head(5)

,answers_question_id,answers_body,answers_author_id
0,332a511f1569444485cf7a7a556a5e54,Hi! You are asking a very interesting quest...,36ff3b3666df400f956f8335cf53e09e
1,eb80205482e4424cad8f16bc25aa2d9c,Hi. I joined the Army after I attended colleg...,2aa47af241bf42a4b874c453f0381bd4
2,eb80205482e4424cad8f16bc25aa2d9c,"Dear Priyanka, Greetings! I have answered th...",cbd8f30613a849bf918aed5c010340be
3,4ec31632938a40b98909416bdd0decff,I work for a global company who values highly...,7e72a630c303442ba92ff00e8ea451df
4,2f6a9a99d9b24e5baa50d40d0ba50a75,I agree with Denise. Every single job I've had...,17802d94699140b0a0d2995f30c034c6


In [0]:
#merged_df = df2.merge(df1,how = 'inner', on = 'answers_question_id')
QA = pd.merge(df2, df1, left_on='answers_question_id',right_on='questions_id')

In [0]:
QA.head(5)

In [0]:
answers_body = QA['answers_body']
questions_body = QA['questions_body']
person_id = QA['answers_author_id']
#person_vec = keras.utils.to_categorical(QA['answers_author_id'],dtype='float32')

In [158]:
person_vec = keras.utils.to_categorical(QA['answers_author_id'],dtype='float32')

ValueError: ignored

In [109]:
answers_body[0]

' Hi!   You are asking a very interesting question.  I am giving you two sites that will give you some of an explanation that may answer your question.   http://mathforum.org/dr.math/faq/faq.why.math.html   http://www.mathworksheetscenter.com/mathtips/mathissoimportant.html   Let me know if this helps '

In [110]:
questions_body[0]

'What  is  a  maths  teacher?   what  is  a  maths  teacher  useful? #college #professor #lecture'

In [0]:
def preprocess(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' + '\n' 
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [0]:
a_train = preprocess(answers_body)
q_train = preprocess(questions_body)

In [113]:
a_train[0]

' Hi    You are asking a very interesting question   I am giving you two sites that will give you some of an explanation that may answer your question    http   mathforum org dr math faq faq why math html   http   www mathworksheetscenter com mathtips mathissoimportant html   Let me know if this helps '

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(a_train)+list(q_train))

In [0]:
a_train = tokenizer.texts_to_sequences(a_train)
q_train = tokenizer.texts_to_sequences(q_train)

In [117]:
q_train[0]

[21, 10, 3, 2287, 269, 21, 10, 3, 2287, 269, 838, 25, 596, 2641]

In [0]:
question_len = 40 
a_train = sequence.pad_sequences(a_train,maxlen=400)
q_train = sequence.pad_sequences(q_train,maxlen=question_len)

In [0]:
#q_train = list(q_train)
#a_train = list(a_train)

In [0]:
q_train = q_train.reshape(q_train.shape[0],q_train.shape[1],1)
a_train = a_train.reshape(a_train.shape[0],a_train.shape[1],1)

In [125]:
a_train.shape

(51123, 400, 1)

In [126]:
q_train.shape

(51123, 40, 1)

In [0]:
import keras 
from keras import layers
from keras import backend as K 
from keras.models import Model
import numpy as np 
from keras import Input
from keras import Sequential
from keras import models

In [0]:
batch_size =16
latent_dim = 2 
question_shape = (40,1)

input_qus = keras.Input(shape=question_shape)
x = layers.LSTM(32,return_sequences=True,input_shape=(40,1))(input_qus)
#x = layers.LSTM((32,return_sequences=True))(x)
x = layers.LSTM((32))(x)
shape_before_flattening = K.int_shape(x)

#x = layers.Flatten()(x)
#x = layers.Dense(10,activation='softmax')(x)
#z_mean = layers.Dense(latent_dim)(x)
z_mean = layers.Dense(10,activation='softmax')(x)
z_log_var = layers.Dense(latent_dim)(x)

que_encoder = Model(input_qus,z_mean)

In [134]:
que_encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 40, 1)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 40, 32)            4352      
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 13,002
Trainable params: 13,002
Non-trainable params: 0
_________________________________________________________________


In [0]:
answer_shape = (400,1)
input_ans = keras.Input(shape=answer_shape)
x = layers.LSTM(64,return_sequences=True,input_shape=(400,1))(input_ans)
x = layers.LSTM((64))(x)
#x = layers.LSTM((64))(x)
#x = layers.Dense(32,activation='softmax')(x)

#z_mean_ans = layers.Dense(latent_dim,name='ans_pool')(x)
z_mean_ans = layers.Dense(10,activation='softmax',name='ans_pool')(x)
z_log_var_ans = layers.Dense(latent_dim)(x)

ans_encoder = Model(input_ans,z_mean_ans,name = 'ans_encoder')

In [0]:
ans_encoder.summary()

In [0]:
concatenated = layers.concatenate([z_mean,z_mean_ans],axis=-1)
#concatenated = (z_mean-z_mean_ans)**2
#combined = layers.Dense(1,input_shape = (1,))(concatenated)
combined = layers.Dense(1,input_shape=(1,))(concatenated)

correlation = Model([input_qus,input_ans],combined)

In [0]:
import keras.backend as K 

def custom_loss(z_mean,z_mean_ans):
  return K.mean(K.square(z_mean-z_mean_ans))

In [0]:
correlation.compile(optimizer = 'rmsprop',loss=custom_loss,metrics=['acc'])

In [140]:
y_true = np.random.random((len(answers_body),1))
correlation.fit([q_train,a_train],y_true,epochs=2,batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
51123/51123 [==============================] - 600s 12ms/step - loss: 0.0845 - acc: 0.0000e+00
Epoch 2/2
51123/51123 [==============================] - 598s 12ms/step - loss: 0.0830 - acc: 0.0000e+00


In [172]:
ans_encoder.predict(a_train[:1])

array([[0.07854894, 0.16068713, 0.06596035, 0.09819147, 0.18369101,
        0.05581289, 0.1001019 , 0.05637481, 0.08534165, 0.11528986]],
      dtype=float32)

In [173]:
que_encoder.predict(q_train[:1])

array([[0.5465924 , 0.01424262, 0.10387947, 0.06904062, 0.05213699,
        0.02471777, 0.0338825 , 0.08189559, 0.02089669, 0.05271532]],
      dtype=float32)

In [0]:
d_input = Input(shape=(10,))
pid = layers.Dense(32,activation='relu')(d_input)
pid = layers.Dense(32,activation='relu')(pid)
pid = layers.Dense(64,activation='relu')(pid)
pid = layers.Dense(100,activation='softmax')(pid)

decoder = Model(d_input,pid)

ans_encoder_output = ans_encoder.get_layer('ans_pool').output

person_id = decoder(ans_encoder_output)

full_model = Model(inputs=ans_encoder.input,outputs=person_id)



In [0]:
keras.utils.plot_model(full_model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [141]:
ans_encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 400, 1)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 400, 64)           16896     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 50,570
Trainable params: 50,570
Non-trainable params: 0
_________________________________________________________________


In [0]:
full_model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])

In [0]:
for layer in full_model.layers[:3]:
    layer.trainable = False

full_model.fit(x_train,pid_true,epochs=20,batch_size=32)